In [1]:
import numpy as np
import pandas as pd

from pandas import Series, DataFrame
import urllib.request
from bs4 import BeautifulSoup

import time
import os

webpage = "https://www.foodnetwork.com/recipes/recipes-a-z/123"

#This returns HTTPResponse object need to read and  
#decode to the see the contents.
html_bytes = urllib.request.urlopen(webpage)

#To decode the bytes to a string this is the HTML of the webpage
#html_string = html_bytes.read().decode("utf-8")

#This is the soup object that will parse th HTML page
soup = BeautifulSoup(html_bytes,"html5lib")

In [27]:
all_links = soup.find_all("a")
href_links = [link.get('href') for link in all_links if link.get('href') is not None]
links=[]
for link in href_links:
    linkStart=link[0:len('//www.foodnetwork.com/recipes/')]
    if(linkStart=='//www.foodnetwork.com/recipes/'):
        links+=[link]
#//www.foodnetwork.com/recipes/
#href_links[href_links.index('//www.foodnetwork.com/recipes/recipes-a-z/xyz')+1:]
links[26:]

['//www.foodnetwork.com/recipes/ina-garten/16-bean-pasta-e-fagioli-1-3753755',
 '//www.foodnetwork.com/recipes/21-apple-pie-recipe-1925900',
 '//www.foodnetwork.com/recipes/21-traditional-hash-brown-potatoes-1963787',
 '//www.foodnetwork.com/recipes/jeff-mauro/1-smore-for-the-road-and-kiddie-smores-recipe-2033028',
 '//www.foodnetwork.com/recipes/1-2-3-jambalaya-3362411',
 '//www.foodnetwork.com/recipes/1-2-3-lasagna-3644969',
 '//www.foodnetwork.com/recipes/1-step-chipotle-hummus-3416679',
 '//www.foodnetwork.com/recipes/alton-brown/10-minute-apple-sauce-recipe2-1938289',
 '//www.foodnetwork.com/recipes/alton-brown/10-minute-apple-sauce-recipe-1950796',
 '//www.foodnetwork.com/recipes/rachael-ray/10-minute-fruit-tart-recipe-1941411',
 '//www.foodnetwork.com/recipes/guy-fieri/10000-island-dressing-with-layered-salad-recipe-1920098',
 '//www.foodnetwork.com/recipes/10-minute-arrabbiata-sauce-2103668',
 '//www.foodnetwork.com/recipes/food-network-kitchen/10-minute-beef-and-beet-salad-wit

In [28]:
def pullLinks(url):
    html_bytes = urllib.request.urlopen(webpage)
    soup1 = BeautifulSoup(html_bytes,"html5lib")
    all_links = soup.find_all("a")
    href_links = [link.get('href') for link in all_links if link.get('href') is not None]
    links=[]
    for link in href_links:
        linkStart=link[0:len('//www.foodnetwork.com/recipes/')]
        if(linkStart=='//www.foodnetwork.com/recipes/'):
            links+=[link]
    links=links[26:]
    return links

In [49]:
def pullRecipe(soup):
    #title
    title=soup.title.string[:soup.title.string.find('|')].rstrip()
    
    #ingredients
    start=soup.get_text().find('"recipeIngredient": [')
    end=start+soup.get_text()[soup.get_text().find('"recipeIngredient": ['):].find(']')
    s=soup.get_text()[start+27:end]
    ingredient=[i for i in s.split('",\n    "')]
    ingredient[-1]=ingredient[-1][:-4]
    
    #directions
    start=soup.get_text().find('"recipeInstructions": [')
    end=start+soup.get_text()[soup.get_text().find('"recipeInstructions": ['):].find(']')
    s=soup.get_text()[start:end]
    if(s.find(',')!=-1):
        directions=[i.rstrip() for i in s[len('"recipeInstructions": '):].split('\n')][1:-1]
    else:
        directions=s[len('"recipeInstructions": '):].rstrip()
    
    #rating
    start=soup.get_text().find('"aggregateRating": {')
    if(start!=-1):
        end=start+soup.get_text()[soup.get_text().find('"aggregateRating": {'):].find('}')
        s=soup.get_text()[start:end]
        reviewCount=int(s.split(',')[2].rstrip()[20:])
        rating=s.split(',')[1].rstrip()[20:]
    else:
        reviewCount=0
        rating=0
    
    #category
    start=soup.get_text().find('"recipeCategory": [')
    end=start+soup.get_text()[soup.get_text().find('"recipeCategory": ['):].find(']')
    s=soup.get_text()[start:end]
    category=[i for i in s[25:].split('",\n    "')]
    category[-1]=category[-1][:-4]
    
    #times
    start=soup.get_text().find('"cookTime": ')
    if(start!=-1):
        end=start+soup.get_text()[soup.get_text().find('"cookTime": '):].find('"totalTime"')
        s=soup.get_text()[start:soup.get_text().find('S',end)+1]
        cookTime=s.split(',')[0].rstrip()[13:-1]
        if(s.find('prepTime')!=-1):
            prepTime=s.split(',')[1].rstrip()[16:-1]
            prepTime= int(prepTime[prepTime.find('T')+1:prepTime.find('H',prepTime.find('T'))])*60+int(prepTime[prepTime.find('H')+1:prepTime.find('M',prepTime.find('H'))])
            totalTime=s.split(',')[2].rstrip()[16:-1]
        else:
            totalTime=s.split(',')[1].rstrip()[16:-1]
            prepTime=0

        cookTime= int(cookTime[cookTime.find('T')+1:cookTime.find('H',cookTime.find('T'))])*60+int(cookTime[cookTime.find('H')+1:cookTime.find('M',cookTime.find('H'))])
        totalTime=int(totalTime[totalTime.find('T')+1:totalTime.find('H',totalTime.find('T'))])*60+int(totalTime[totalTime.find('H')+1:totalTime.find('M',totalTime.find('H'))])
    else:
        prepTime=None
        cookTime=None
        totalTime=None
        
    #create series
    columns=[ingredient,category,rating,reviewCount,directions,cookTime,prepTime,totalTime]
    labels=['ingredients','categories','rating','review count','directions','cook time','prep time','total time']
    s=Series(columns,index=labels)
    
    return (title,s)
    


In [51]:
links=pullLinks('https://www.foodnetwork.com/recipes/recipes-a-z/123')
recipes=DataFrame()
i=0
for link in links:
    if(i==30):
        break
    i+=1
    html_bytes = urllib.request.urlopen('https:'+link)
    recipeSoup = BeautifulSoup(html_bytes,"html5lib")
    if(recipeSoup.get_text().find('"aggregateRating": {')!=-1):
        temp=pullRecipe(recipeSoup)
        recipes[temp[0]]=temp[1]
        
recipes=recipes.transpose()
recipes

#write to save files every 100 or so https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_pickle.html

,ingredients,categories,rating,review count,directions,cook time,prep time,total time
"""16 Bean"" Pasta e Fagioli Recipe","[1 (1-pound) bag 16 Bean Soup Mix, 2 tablespoo...","[Italian, Soup, Beans and Legumes, Main Dish]",5,3,"[ ""The day before you plan to make the soup...",4,0,610
1 S'more for the Road and Kiddie S'mores Recipe,"[Nonstick canola oil spray, 1 pound marshmallo...","[S'mores, Chocolate, Bourbon, Dessert]",4.5,2,"[ ""Coat a large microwave-safe bowl with no...",60,0,120
1-Step Chipotle Hummus Recipe,[1 (16-ounce) container store-bought hummus (a...,"[Hummus Recipes, Vegetable, Recipes for Partie...",1,2,"[""Mix together hummus and TABASCO&reg; Chipotl...",None,None,None
10 Minute Apple Sauce Recipe,"[3 Golden Delicious apples, peeled, cored, and...","[Easy Side Dish Recipes, Easy, Side Dish, Sauc...",5,3,"[ ""In a sealable microwave-safe container, ...",None,None,None
10 Minute Fruit Tart Recipe,[1 box vanilla or lemon flavor instant pudding...,"[Easy Dessert Recipes, Dessert, Easy, Fruit De...",3.9,37,[],None,None,None
"10,000 Island Dressing with Layered Salad Recipe","[1 cup mayonnaise, 3 tablespoons mustard, pref...","[Beet Salad, Beet, Salad Recipes, Vegetable, B...",4.4,7,"[ ""Dressing: In a glass bowl, add the mayon...",60,25,105
"10-Minute Chicken, Corn and Kimchi Ramen Recipe","[4 cups low-sodium chicken broth, Three 3-ounc...","[Pasta Recipes, Noodles, Chicken, Poultry, Cor...",5,3,"[ ""Combine the chicken broth, 1 cup water, ...",8,2,10
10-Minute Green Gazpacho and Smoky Quesadilla Recipe,"[1/2 cup sour cream, 1 English cucumber roughl...","[Quesadilla, Beans and Legumes, Cheese, Tomato...",4,1,"[ ""Preheat the broiler with a rack set abou...",5,10,15
"10-Minute Ham, White Bean and Kale Salad Recipe","[Two 15-ounce cans white beans, rinsed and dra...","[Easy, Salad Recipes, Beans and Legumes, Ham, ...",4,1,"[ ""Meanwhile, whisk together the red wine v...",5,5,10
10-Minute White Bean Soup with Toasted Cheese and Tomato Recipe,"[1/2 cup dry white wine, 2 sprigs fresh rosema...","[Tomato Soup, Vegetable Soup, Soup, Tomato, Ve...",4,9,"[ ""Position a rack about 4 inches from the ...",6,4,10


In [120]:
s=""
for index, row in recipes.iterrows():
    s+=((row.name+' '+' '.join(row['ingredients'])+' '+' '.join(row['categories'])).lower())
s=s.replace('\xa0',"")
r=['1','2','3','4','5','6','7','8','9','0','cups','cup','tablespoons','tablespoon','teaspoons','teaspoon',
   '/','(',')','"',',','-','\n',':','½',' or ',' if ',' at ',' oz ','ounces','ounce',' for ',' as ','good',' to ','plus',' in ','half',
   'recipe',' we ',' and ',' easy ', ' one ']
for i in r:
    s=s.replace(i," ")
s=s.split(" ")
s.remove('')

dictionary=dict()
for i in s:
    if(len(i)>1):
        dictionary[i]=0
print(dictionary)

{'juiced': 0, 'purpose': 0, 'rice': 0, 'mayonnaise': 0, 'chile': 0, 'white': 0, 'baby': 0, 'seedless': 0, 'aisle': 0, 'tubettini': 0, 'stackable': 0, 'layered': 0, 'turkey': 0, 'medium': 0, 'pre': 0, 'clove': 0, 'beet': 0, 'into': 0, 'dish': 0, 'mix': 0, 'soup': 0, 'verde': 0, 'crust': 0, 'soy': 0, 'cinnamon': 0, 'pickles': 0, 'crushed': 0, 'piquante': 0, 'black': 0, 'tart': 0, 'iceberg': 0, 'blackberries': 0, 'brunch': 0, 'oil': 0, 'occasionally': 0, 'over': 0, 'handful': 0, 'skinless': 0, 'cooking': 0, 'stems': 0, 'covered': 0, 'edamame': 0, 'pancetta': 0, 'thick;': 0, 'vermouth': 0, 'homemade': 0, 'ripe': 0, 'low': 0, 'boneless': 0, 'tailgate': 0, 'plum': 0, 'discarded': 0, 'pickled': 0, 'three': 0, 'instructions': 0, 'roasting': 0, 'spray': 0, 'mozzarella': 0, 'cucumber': 0, 'grape': 0, 'focaccia': 0, 'onion': 0, 'removed': 0, 'poultry': 0, 'salmon': 0, 'blackberry': 0, 'cream': 0, 'wonton': 0, 'refried': 0, 'on': 0, 'cooled': 0, 'arugula': 0, 'instant': 0, 'mint': 0, 'sourdough': 

In [62]:
(recipes.index[0]+' '+' '.join(recipes['ingredients'][0])+' '+' '.join(recipes['categories'][0])).lower()

'"16 bean" pasta e fagioli recipe 1 (1-pound) bag 16 bean soup mix 2 tablespoons good olive oil, plus extra for serving 6 ounces pancetta, 1/4-inch-diced 1 large onion, chopped 1 tablespoon minced garlic (3 cloves) 1/2 teaspoon crushed red pepper flakes 1 (28-ounce) can crushed tomatoes 1 cup dry red wine 4 to 6 cups good chicken stock, preferably homemade kosher salt and freshly ground black pepper 1 cup miniature pasta, such as ditalini or tubettini 1/2 cup freshly grated italian parmesan cheese, plus extra for serving 1 tablespoon good red wine vinegar julienned fresh basil leaves, for serving italian soup beans and legumes main dish'